In [24]:
import pandas as pd
import numpy as np
import re

In [40]:
df = pd.read_csv("panel_2000_2010.csv")

df.head()

,id,year,region,nervous,calm,down,happy,depressed,income,msa,urban_rural,prev_region,moved,mh_index
0,1.0,2000,1.0,3.0,3.0,3.0,3.0,3.0,128000.0,3.0,1.0,NaN,0,3.000000
1,1.0,2001,1.0,NaN,NaN,NaN,NaN,NaN,177000.0,3.0,1.0,1.0,0,NaN
2,1.0,2002,1.0,1.0,4.0,2.0,3.0,2.0,128000.0,3.0,1.0,1.0,0,1.666667
3,1.0,2003,1.0,NaN,NaN,NaN,NaN,NaN,185000.0,3.0,1.0,1.0,0,NaN
4,1.0,2004,1.0,1.0,4.0,1.0,4.0,1.0,200000.0,3.0,1.0,1.0,0,1.000000


In [45]:
# Sunlight mapping based on census region
sun_map = {
    1.0: 3.5,   # Northeast
    2.0: 4.0,   # North Central
    3.0: 5.2,   # South
    4.0: 6.0,   # West
    -5.0: None  # NIU / missing
}

# Add sunlight variable
df["sunlight"] = df["region"].map(sun_map)

# Add lagged sunlight (previous year's sunlight)
df["sunlight_lag"] = df.groupby("id")["sunlight"].shift(1)

# Change in sunlight after moving
df["sunlight_change"] = df["sunlight"] - df["sunlight_lag"]

# Only count positive sunlight increases (from moving to sunnier region)
df["sunlight_increase"] = (df["sunlight_change"] > 0).astype(int)

# Replace all -5 values with NaN for the relevant columns
cols_to_fix = ["nervous","calm","down","happy","depressed","income","sunlight"]


for c in cols_to_fix:
    df[c] = df[c].replace(-5, np.nan)

df.head()

df["mh_index"] = df[["nervous","calm","down","happy","depressed"]].mean(axis=1)
df["sunlight_move_interaction"] = df["sunlight_change"] * df["moved"]

df



,id,year,region,nervous,calm,down,happy,depressed,income,msa,urban_rural,prev_region,moved,mh_index,sunlight,sunlight_lag,sunlight_change,sunlight_increase,sunlight_move_interaction
0,1.0,2000,1.0,3.0,3.0,3.0,3.0,3.0,128000.0,3.0,1.0,NaN,0,3.0,3.5,NaN,NaN,0,NaN
1,1.0,2001,1.0,NaN,NaN,NaN,NaN,NaN,177000.0,3.0,1.0,1.0,0,NaN,3.5,3.5,0.0,0,0.0
2,1.0,2002,1.0,1.0,4.0,2.0,3.0,2.0,128000.0,3.0,1.0,1.0,0,2.4,3.5,3.5,0.0,0,0.0
3,1.0,2003,1.0,NaN,NaN,NaN,NaN,NaN,185000.0,3.0,1.0,1.0,0,NaN,3.5,3.5,0.0,0,0.0
4,1.0,2004,1.0,1.0,4.0,1.0,4.0,1.0,200000.0,3.0,1.0,1.0,0,2.2,3.5,3.5,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98819,NaN,2006,4.0,4.0,2.0,3.0,2.0,3.0,35000.0,3.0,1.0,NaN,0,2.8,6.0,NaN,NaN,0,NaN
98820,NaN,2007,4.0,NaN,NaN,NaN,NaN,NaN,17500.0,3.0,1.0,NaN,0,NaN,6.0,NaN,NaN,0,NaN
98821,NaN,2008,NaN,4.0,2.0,3.0,3.0,3.0,25000.0,3.0,1.0,NaN,0,3.0,NaN,NaN,NaN,0,NaN
98822,NaN,2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,NaN,0,NaN,NaN,NaN,NaN,0,NaN


In [46]:
df.to_csv("sunlight_region.csv", index=False)
df.head()

,id,year,region,nervous,calm,down,happy,depressed,income,msa,urban_rural,prev_region,moved,mh_index,sunlight,sunlight_lag,sunlight_change,sunlight_increase,sunlight_move_interaction
0,1.0,2000,1.0,3.0,3.0,3.0,3.0,3.0,128000.0,3.0,1.0,NaN,0,3.0,3.5,NaN,NaN,0,NaN
1,1.0,2001,1.0,NaN,NaN,NaN,NaN,NaN,177000.0,3.0,1.0,1.0,0,NaN,3.5,3.5,0.0,0,0.0
2,1.0,2002,1.0,1.0,4.0,2.0,3.0,2.0,128000.0,3.0,1.0,1.0,0,2.4,3.5,3.5,0.0,0,0.0
3,1.0,2003,1.0,NaN,NaN,NaN,NaN,NaN,185000.0,3.0,1.0,1.0,0,NaN,3.5,3.5,0.0,0,0.0
4,1.0,2004,1.0,1.0,4.0,1.0,4.0,1.0,200000.0,3.0,1.0,1.0,0,2.2,3.5,3.5,0.0,0,0.0


In [ ]:
from linearmodels.panel import PanelOLS
import statsmodels.api as sm

# Load your data
df = pd.read_csv("sunlight_region.csv")

# Set panel index
df = df.set_index(["id", "year"])

# Drop rows with missing mh_index or sunlight_change
df_clean = df.dropna(subset=["mh_index", "sunlight_change", "sunlight_move_interaction"])

# Create regression model
model = PanelOLS.from_formula(
    "mh_index ~ sunlight_change + moved + sunlight_move_interaction + income + msa + urban_rural + EntityEffects + TimeEffects",
    data=df_clean
)


results = model.fit(cov_type="clustered", cluster_entity=True)
print(results)

/opt/anaconda3/envs/ds30/lib/python3.12/site-packages/linearmodels/panel/model.py:1258: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


ValueError: exog does not have full column rank. If you wish to proceed with model estimation irrespective of the numerical accuracy of coefficient estimates, you can set check_rank=False.